In [3]:
import os, sys
import django
from datetime import datetime
sys.path.append('../') # add path to project root dir
import pandas as pd
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
import json
from django.db.models.signals import post_save, pre_save
from tqdm import tqdm
from flourish_caregiver.models import GeneralSymptoms

flourish_caregiver_file = open('flourish_caregiver_medicalhistory.json')

data_list = json.load(flourish_caregiver_file)
errors = []

pre_save.disconnect(sender=MedicalHistory)
post_save.disconnect(sender=MedicalHistory)

for data_dict in tqdm(data_list):
    model_cls = django_apps.get_model(data_dict['model'])

    try:
        model_obj = model_cls.objects.get(pk = data_dict['pk'])
    except model_cls.DoesNotExist:
        pass
    else:
        current_symptoms_value = data_dict['fields'].get('current_symptoms', '')

        general_symptoms_objs = GeneralSymptoms.objects.filter(
            short_name = current_symptoms_value)

        model_obj.current_symptoms.set(general_symptoms_objs)

        try:
            model_obj.save()
        except Exception as e:
            subject_identifier = model_obj.maternal_visit.subject_identifier
            errors.append({
                'subject_identifier': subject_identifier,
                'exception': e})





FileNotFoundError: [Errno 2] No such file or directory: 'flourish_caregiver_medicalhistory.json'

In [ ]:
import concurrent.futures, multiprocessing, threading
from tqdm import tqdm




import json
from django.apps import apps as django_apps
from django.db.models.signals import pre_save, post_save
from flourish_child.models import *
import os



num_cores = os.cpu_count()

recommended_threads = min(8, num_cores)  # For example, limit to 8 threads

# Using ThreadPoolExecutor to parallelize the for loop
with concurrent.futures.ThreadPoolExecutor(max_workers=recommended_threads) as executor:
    
    print('=== querysets to be updated====')

    # Map the function to the items, running in parallel
    _ = tqdm(executor.map(update_queryset, data_list), total=len(data_list))


In [ ]:
def populate():
    for data_dict in tqdm(data_list):
    model_cls = django_apps.get_model(data_dict['model'])

    try:
        model_obj = model_cls.objects.get(pk = data_dict['pk'])
    except model_cls.DoesNotExist:
        pass
    else:
        current_symptoms_value = data_dict['fields'].get('current_symptoms', '')
        current_medications_value = data_dict['fields'].get('current_medications', '')

        general_symptoms_objs = GeneralSymptoms.objects.filter(
            short_name = current_symptoms_value)

        medications_objs = Medications.objects.filter(
        short_name = current_medications_value)

        model_obj.current_symptoms.set(general_symptoms_objs)
        model_obj.current_medications.set(medications_objs)

        try:
            model_obj.save()
        except Exception as e:
            subject_identifier = model_obj.child_visit.subject_identifier
            errors.append({
                'subject_identifier': subject_identifier,
                'exception': e})
